In [ ]:
from heapq import heappush, heappop
from itertools import product

import matplotlib.pyplot as plt
import numpy as np
from pydrake.all import (
    DiagramBuilder,
    AddMultibodyPlantSceneGraph, Parser,
    RigidTransform, RotationMatrix, Role, Box,
    ConnectPlanarSceneGraphVisualizer,
    MathematicalProgram, MosekSolver,
)

import os
os.environ['MOSEKLM_LICENSE_FILE'] = '/Users/russt/mosek/mosek.lic'

# Construct a simple obstacle field using box geometry in Drake's geometry engine (SceneGraph).

# URDF (Universal Robot Description Format) for the box
box_urdf = """
<?xml version="1.0"?>
<robot name="box">
  <link name="base">
    <visual>
      <geometry><box size="1 1 1" /></geometry>
      <material><color rgba="0 0 0.7 1" /></material>
    </visual>
    <collision>
      <geometry><box size="1 1 1" /></geometry>
    </collision>
  </link>
</robot>
"""

def make_obstacles():
    builder = DiagramBuilder()
    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.1)
    parser = Parser(plant)

    #num_obstacles = 1
    #xytheta = np.random.random((num_obstacles, 3)) @ np.diag([2.5, 2.5, 1]) + np.tile([1, 1, 0],(num_obstacles,1))
    xytheta = np.array([[1, 1, 0.1],
                        [1.5, 3, 0.0],
                        [3.2, 1.8, 0.0]])
    for i in range(xytheta.shape[0]):
        box = parser.AddModelFromString(box_urdf, "urdf", f"box{i}")
        X_WB = RigidTransform(RotationMatrix.MakeZRotation(xytheta[i,2]), [xytheta[i,0], xytheta[i,1], 0])
        plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("base", box), X_WB)

    plant.Finalize()

    T_VW = np.eye(4)[[0,1,3],:]
    vis = ConnectPlanarSceneGraphVisualizer(
        builder, scene_graph, T_VW=T_VW, xlim=[0, 4], ylim=[0, 4])

    diagram = builder.Build()

    return diagram, scene_graph, vis

diagram, scene_graph, vis = make_obstacles()
solver = MosekSolver()
#context = diagram.CreateDefaultContext()
#diagram.Publish(context)

def shortest_path(start, goal):
    context = diagram.CreateDefaultContext()
    sg_context = scene_graph.GetMyContextFromRoot(context)
    query_object = scene_graph.get_query_output_port().Eval(sg_context)
    inspector = query_object.inspector()

    start = np.asarray(start)
    goal = np.asarray(goal)
    As = []   # for each object, Ax<=b
    bs = []

    for fid in inspector.all_frame_ids():
        for gid in inspector.GetGeometries(fid, Role.kProximity):
            shape = inspector.GetShape(gid)
            if not isinstance(shape, Box):
                continue
            X_GW = query_object.GetPoseInWorld(gid).inverse()
            # For 2D, I've assumed rotations are only about z
            assert np.isclose(X_GW.rotation().multiply([0, 0, 1])[2], 1.0)
            # Constraints (in 3D) are one of
            #   p_GA = X_GW * p_A <= -[width, depth, height]/2
            #   p_GA >= [width, depth, height]/2
            # intersected with the bounding box
            wdh = np.array([shape.width(), shape.depth(), shape.height()])
            C = np.vstack((X_GW.rotation().matrix(), -X_GW.rotation().matrix()))
            d = np.hstack((-wdh/2.0 - X_GW.translation(),
                           -wdh/2.0 + X_GW.translation()))
            # Chop it down to 2D
            C = C[:,[0,1]]
            As.append(C[[0,1,3,4]])
            bs.append(d[[0,1,3,4]])

    # Returns the "neighbors" from a boolean vector corresponding to the faces of any object.
    # Currently hard-coded for the As, bs created above. (TODO: generalize this)
    def neighbor_activations(zi):
        assert zi.size == 4
        nz = np.nonzero(zi)[0]
        zs = np.zeros((2,4))
        if len(nz)==1:
            # Case 1: One active face.  Can enable either neighbor.
            zs[0, nz] = True
            zs[0, (nz+1) % num_faces] = True
            zs[1, nz] = True
            zs[1, (nz-1) % num_faces] = True
            return zs
        elif len(nz)==2:
            # Case 2: Two active faces.  Can disable either active face.
            zs[0, nz[0]] = True
            zs[1, nz[1]] = True
            return zs
        raise Exception("I should never have more than two active faces")


    num_obstacles = len(bs)
    num_faces = len(bs[0])
    z_start = np.array([A @ start <= b for (A, b) in zip(As, bs)])

    def make_polytope(z):
        n = np.count_nonzero(z)
        A = np.zeros((n, 2))
        b = np.zeros((n, 1))
        count = 0
        for o in range(num_obstacles):
            for f in np.nonzero(z[o,:])[0]:
                A[count,:] = As[o][f,:]
                b[count] = bs[o][f]
                count += 1
        assert count == n
        return A, b

    def vertex_enumeration(A,b):
        # brute force vertex enumeration
        vertices = []
        for i in range(len(b)):
            for j in range(i+1,len(b)):
                try:
                    x = np.linalg.solve(A[[i,j],:],b[[i,j]])
                except:
                    continue
                if np.alltrue(A @ x <= b):
                    vertices.append(x)
        if not vertices:
            return None
        v = np.asarray(vertices)
        v2 = v[np.lexsort((v[:,0], v[:,1])),:]
        return v2

    def AddTwoNormCost(prog, e):
        z = prog.NewContinuousVariables(1, 'z')
        prog.AddLorentzConeConstraint(z[0], e.dot(e))
        prog.AddLinearCost([1], 0, z)

    class Node(object):
        def __init__(self, z, parent):
            self.z = z
            self.parent = parent
            self.lower_bound = None
            self.xsol = None
            self.A, self.b = make_polytope(z)
            A = np.vstack((self.A, np.eye(2), -np.eye(2)))
            b = np.concatenate((self.b, [4, 4, 0, 0]),axis=None)
            self.vertices = vertex_enumeration(A,b)

        def __lt__(self, other):
            return self.lower_bound < other.lower_bound

        def __eq__(self, other):
            return self.lower_bound == other.lower_bound

        def __gt__(self, other):
            return self.lower_bound > other.lower_bound

        def get_polytope(self):
            return self.A, self.b

        def same_region(self, other):
            # TODO: is there a better way to do this?
            return np.array_equal(self.vertices, other.vertices)

    def plot_region(ax, node):
        # TODO: take convex hull https://www.cs.jhu.edu/~misha/Spring16/06.pdf
        # For now, i can hack it:
        vertices = node.vertices[[0,1,3,2],:] if node.vertices.shape[0]==4 else node.vertices
        ax.fill(vertices[:,0], vertices[:,1])

    def old_heuristic(node):
        A, b = node.get_polytope()
        prog = MathematicalProgram()
        x = prog.NewContinuousVariables(2, 'x')
        prog.AddLinearConstraint(A, b-np.inf, b, x)
        AddTwoNormCost(prog, x-goal)
        result = solver.Solve(prog)
        return result.get_optimal_cost() if result.is_success() else np.inf

    def plan(node, plot=False):
        # Returns cost, xsol, contains_goal
        # Plans from the start through the tree defined by node (and it's parents), then hops in a straight line to the goal
        # If contains_goal = False, then cost is a lower bound on the true cost for this branch
        # If contains_goal = True, then plan solves the full problem, and the cost is also an upper bound on the optimal cost (for any path)
        prog = MathematicalProgram()
        current = node
        x = []
        #Q = np.block([[np.eye(2), -np.eye(2)],[-np.eye(2), np.eye(2)]])
        if plot:
            fig, ax = plt.subplots()
            ax.set_xlim([0,4])
            ax.set_ylim([0,4])
        count=0
        while current:
            x.append(prog.NewContinuousVariables(2,f'x{count}'))
            A, b = current.get_polytope()
            if plot:
                plot_region(ax, current)
            prog.AddLinearConstraint(A, b-np.inf, b, x[-1])
            if len(x)>1:
#                prog.AddQuadraticCost(Q, np.zeros((4,1)), np.hstack((x[-1],x[-2])))
                AddTwoNormCost(prog, x[-1]-x[-2])
            current = current.parent
            count += 1
        prog.AddBoundingBoxConstraint(start, start, x[-1])
        contains_goal = np.all(node.A @ goal <= node.b)
        if contains_goal:
            prog.AddBoundingBoxConstraint(goal, goal, x[0])
        else:
            AddTwoNormCost(prog, x[0]-goal)
        result = solver.Solve(prog)
        #print(result.get_solution_result())
        assert result.is_success()
        xsol = np.vstack((goal, result.GetSolution(np.array(x))))
        if plot:
            ax.plot(xsol[:,0],xsol[:,1],'r-*')
            plt.show()
        return result.get_optimal_cost(), xsol, contains_goal

    queue = []

    # Add all permutations of "one face per obstacle" regions that contain the start.
    # Regions with more faces active are only more constrained (higher heuristic cost to go).
    nz = (np.nonzero(z_start[o,:])[0] for o in range(num_obstacles))
    for p in product(*nz):
        z = np.zeros((num_obstacles,num_faces))
        for i,pi in enumerate(p):
            z[i,pi] = True
        node = Node(z=z, parent=None)
        # TODO: Handle case where goal is in the starting region
        node.lower_bound,node.xsol,_ = plan(node)
        heappush(queue, node)

#    fig, ax = plt.subplots(2,1)
    upper_bound = np.inf
    count = 0
    x_solution = None
    tol = 1e-5
    while queue:
        node = heappop(queue)
        print([node.lower_bound, upper_bound])
        if node.lower_bound >= upper_bound + tol:
#            fig, ax = plt.subplots()
            vis.ax.plot(x_solution[:,0], x_solution[:,1],'g-*')
            break
        # add all children.  todo: implement this in a more general way
        for o in range(num_obstacles):
            neighbors = neighbor_activations(node.z[o,:])
            for n in range(neighbors.shape[0]):
                z = node.z.copy()
                z[o,:] = neighbors[n,:]
                child = Node(z=z, parent=node)
                if child.vertices is None:
                    continue
                ancestor = node
                loop = False
                while ancestor:
                    if ancestor.same_region(child):
                        loop = True
                        break
                    ancestor = ancestor.parent
                if loop:
                    continue
                cost, xsol, to_goal = plan(child, plot=False)
                child.lower_bound = cost
                child.xsol = xsol  # TODO: Remove this?
                if to_goal:
                    if cost < upper_bound:
                        upper_bound = cost
                        x_solution = xsol
                else:
                    if child.lower_bound < node.lower_bound - tol:
                        fig, ax = plt.subplots(1,2)
                        plot_region(ax[0], node)
                        ax[0].plot(node.xsol[:,0],node.xsol[:,1],'-*')
                        plot_region(ax[1], child)
                        ax[1].plot(child.xsol[:,0],child.xsol[:,1],'-*')
                        for a in ax:
                            a.set_xlim(0, 4)
                            a.set_ylim(0, 4)
                    assert child.lower_bound >= node.lower_bound - tol
                    if child.lower_bound < upper_bound:
                        heappush(queue, child)
        count += 1

    diagram.Publish(context)


shortest_path(start=[0.2, 0.2], goal=[3.5, 3.5])

The rest is just little code snippets from during development...

In [ ]:
prog = MathematicalProgram()
x = prog.NewContinuousVariables(2,'x')
y = prog.NewContinuousVariables(2,'y')
print((x-y).dot(x-y))
Q = np.block([[np.eye(2), -np.eye(2)],[-np.eye(2), np.eye(2)]])
xy = np.hstack((x,y))
print(xy.T @ Q @ xy)


In [ ]:
from itertools import product
z = np.array([[True, True, False, False], [True, False, False, False], [False, True, True, True]])

nz = (np.nonzero(z[o,:])[0] for o in range(3))
print(nz)
for p in product(*nz):
    z = np.zeros((3,4))
    for i,pi in enumerate(p):
        z[i,pi] = True
    print(z)